In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 144
from itertools import product

from gettsim import get_policies_for_date
from gettsim import compute_taxes_and_transfers

# Kindergeld und Kinderfreibetrag

Nachdem Sie sich in der Selbstlernphase mit GETTSIM vertraut gemacht haben, werden wir es erneut nutzen, 
um die Mechanik von Kindergeld und Kinderfreibetrag besser zu verstehen.

Wir werden Ehepartner ohne Kinder, mit einem Kind und mit zwei Kindern vergleichen. Die Einkommen beider
Ehepartner sind identisch. 

Unsere Daten benötigen dasselbe Format, das wir in der vergangenen Woche verwendet haben. Eine Zeile
pro Haushaltsmitglied.

Wir beginnen mit einem DataFrame mit einer Zeile pro Haushalt, den wir dann vervierfachen und entsprechend
für die einzelnen Haushaltsmitglieder anpassen.

## Datensatzerstellung 1: Grundlage, die für alle Haushaltsmitglieder gleich sind

In [2]:
min_einkommen = 1500
max_einkommen = 8000
anzahl_schritte = 131

df_grundlage = pd.DataFrame(index=pd.RangeIndex(anzahl_schritte))

# Haushalts-ID und Steuersubjekt-ID: In unserem Beispiel kein Unterschied
df_grundlage["hh_id"] = df_grundlage.index
df_grundlage["tu_id"] = df_grundlage.index
# Bruttolohn ist am einfachsten, jetzt zu erstellen und für Kinder auf 0 zu setzen.
df_grundlage["bruttolohn_m"] = np.linspace(min_einkommen, max_einkommen, anzahl_schritte)
# Bruttolohn, Vermögen, private Krankenversicherung sind dasselbe für alle Haushaltsmitglieder
df_grundlage["vermögen_hh"] = 100
df_grundlage["prv_krankenv"] = False

df_grundlage

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv
0,0,0,1500.0,100,False
1,1,1,1550.0,100,False
2,2,2,1600.0,100,False
3,3,3,1650.0,100,False
4,4,4,1700.0,100,False
...,...,...,...,...,...
126,126,126,7800.0,100,False
127,127,127,7850.0,100,False
128,128,128,7900.0,100,False
129,129,129,7950.0,100,False


## Datensatzerstellung 2: Vervierfachung für 2 Erwachsene und 2 Kindern

In [3]:
df_2e_2k = df_grundlage.append(df_grundlage).append(df_grundlage).append(df_grundlage).sort_index()
df_2e_2k.index = pd.RangeIndex(len(df_2e_2k), name="p_id")
df_2e_2k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv
p_id,,,,,
0,0,0,1500.0,100,False
1,0,0,1500.0,100,False
2,0,0,1500.0,100,False
3,0,0,1500.0,100,False
4,1,1,1550.0,100,False
...,...,...,...,...,...
519,129,129,7950.0,100,False
520,130,130,8000.0,100,False
521,130,130,8000.0,100,False


## Datensatzerstellung 3: Anpassung der Werte, die sich über Haushaltsmitglieder unterscheiden

Nicht alle Werte sind überall nötig. Wir nutzen den Modulus-Operator %, um einzelne Zeilen anzusteuern.

Zunächst eine kurze Demonstration desselben:

In [4]:
demo = df_2e_2k.loc[:10][["tu_id"]].copy()
display(demo)
demo['modulus'] = -9
for i in demo.index:
    demo.loc[i, "modulus"] = i % 4
display(demo)

,tu_id
p_id,
0,0
1,0
2,0
3,0
4,1
5,1
6,1
7,1
8,2


,tu_id,modulus
p_id,,
0,0,0
1,0,1
2,0,2
3,0,3
4,1,0
5,1,1
6,1,2
7,1,3
8,2,0


Wir nutzen in der folgenden Zelle aus, dass unsere Daten immer vier Zeilen pro Haushalt mit einem
Einkommen haben.

Die ersten beiden Zeilen eines Haushalts sind die Erwachsenen, danach kommen die beiden Kinder.
Für GETTSIM ist die Reihenfolge egal, dies ist lediglich unsere Konvention.

In [5]:
# Erwachsene
for modulus in 0, 1:
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alter"] = 50
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "hat_kinder"] = True
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "kind"] = False

    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "selbstständig"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "eink_selbst_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "ges_rente_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "vermiet_eink_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "kapital_eink_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "prv_rente_beitr_m"] = 0

    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "wohnort_ost"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "jahr_renteneintr"] = 2100
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "betreuungskost_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "in_ausbildung"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "behinderungsgrad"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "arbeitsstunden_w"] = 40
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alleinerziehend"] = False

# Kinder
for modulus in 2, 3:
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "bruttolohn_m"] = np.nan
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "kind"] = True
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "hat_kinder"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alleinerziehend"] = False
    

# Nur erstes Kind: 10 Jahre alt
modulus = 2
df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alter"] = 10

# Nur zweites Kind: 5 Jahre alt
modulus = 3
df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alter"] = 5

# Einige Zellen benötigen explizit eine Boole'sche Variable
for col in "kind", "hat_kinder", "alleinerziehend":
    df_2e_2k[col] = df_2e_2k[col].astype('bool')
df_2e_2k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv,alter,hat_kinder,kind,selbstständig,eink_selbst_m,...,vermiet_eink_m,kapital_eink_m,prv_rente_beitr_m,wohnort_ost,jahr_renteneintr,betreuungskost_m,in_ausbildung,behinderungsgrad,arbeitsstunden_w,alleinerziehend
p_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1500.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
1,0,0,1500.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
2,0,0,NaN,100,False,10.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,0,0,NaN,100,False,5.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,1,1,1550.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,129,129,NaN,100,False,5.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
520,130,130,8000.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
521,130,130,8000.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False


## Datensatzerstellung 4: Kürzung der Haushalte um ein Kind bzw. beide Kinder

Hierfür nutzen wir aus, dass wir den beiden Kindern überall dieselben Alter gegeben haben.

In [6]:
df_2e_1k = df_2e_2k.query("alter > 5").copy()
df_2e_1k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv,alter,hat_kinder,kind,selbstständig,eink_selbst_m,...,vermiet_eink_m,kapital_eink_m,prv_rente_beitr_m,wohnort_ost,jahr_renteneintr,betreuungskost_m,in_ausbildung,behinderungsgrad,arbeitsstunden_w,alleinerziehend
p_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1500.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
1,0,0,1500.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
2,0,0,NaN,100,False,10.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,1,1,1550.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
5,1,1,1550.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,129,129,7950.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
518,129,129,NaN,100,False,10.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
520,130,130,8000.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False


In [7]:
df_2e_0k = df_2e_2k.query("alter > 10").copy()
df_2e_0k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv,alter,hat_kinder,kind,selbstständig,eink_selbst_m,...,vermiet_eink_m,kapital_eink_m,prv_rente_beitr_m,wohnort_ost,jahr_renteneintr,betreuungskost_m,in_ausbildung,behinderungsgrad,arbeitsstunden_w,alleinerziehend
p_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1500.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
1,0,0,1500.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
4,1,1,1550.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
5,1,1,1550.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
8,2,2,1600.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,128,128,7900.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
516,129,129,7950.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
517,129,129,7950.0,100,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False


## Gruppieren der Datensätze in einem dict

Da alle Datensätze die gleiche Struktur haben, empfiehlt es sich, sie in einem Dictionary
zu gruppieren. Der key ist die Zahl der Kinder, der value der eben erstellte Datensatz.

Im nächsten Schritt können wir dann einfach darüber schleifen, anstatt den Aufruf von
GETTSIM dreimal zu programmieren.

In [8]:
daten_nach_kindern = {
    0: df_2e_0k,
    1: df_2e_1k,
    2: df_2e_2k,
}

## Anwendung von GETTSIM für Kindergeld und Steuern

Zunächst setzen wir wieder das Steuersystem für 2020 auf.

In [9]:
params_dict, policy_func_dict = get_policies_for_date("2020")

Nun rufen wir GETTSIM auf. Als Rückgabe benötigen wir die Identifizierungsnummer für das
Steuersubjekt, das Kindergeld, sowie Einkommensteuer und Solidaritätszuschlag.

Funktionen und Parameter übernehmen wir direkt aus dem Steuersystem.

In [10]:
ergebnisse_nach_kindern = {}
for n_kinder, daten in daten_nach_kindern.items():
    ergebnisse_nach_kindern[n_kinder] = compute_taxes_and_transfers(
        daten,
        user_functions=policy_func_dict,
        params=params_dict,
        targets=[
                "kindergeld_m_tu",
                "eink_st_tu",
            ],
    )

In [11]:
ergebnisse_nach_kindern[1]

,kindergeld_m_tu,eink_st_tu
p_id,,
0,204,1611.323123
1,204,1611.323123
2,204,1611.323123
4,204,1843.015567
5,204,1843.015567
...,...,...
517,0,48631.495800
518,0,48631.495800
520,0,49135.495800


## Berechnung jährlicher Werte auf Steuersubjektsebene, schöne Variablennamen

Wie Sie an dem DataFrame oberhalb dieser Zelle sehen, sind die Werte für
alle Mitglieder einer Steuereinheit repliziert. 

Außerdem ist das Kindergeld auf monatlicher Ebene, für die
Vergleichbarkeit brauchen wir aber jährliche Werte.

Schließlich wollen wir gleich über die Zahl der Kinder hinweg vergleichen,
wenn wir die Datensätze zusammenfügen, brauchen wir unterschiedliche
Variablennamen.

In [12]:
for n_kinder in ergebnisse_nach_kindern.keys():
    # Jährliches Kindergeld auf Ebene der Steuersubjekte
    ergebnisse_nach_kindern[n_kinder]["kindergeld_tu"] = ergebnisse_nach_kindern[n_kinder]["kindergeld_m_tu"] * 12
    # Lösche monatliche Werte
    ergebnisse_nach_kindern[n_kinder] = ergebnisse_nach_kindern[n_kinder].drop("kindergeld_m_tu", axis=1)
    # Behalte nur eine Zeile pro Steuersubjekt
    ergebnisse_nach_kindern[n_kinder] = ergebnisse_nach_kindern[n_kinder].drop_duplicates("tu_id").set_index("tu_id")
    #
    if n_kinder == 0:
        zusatz = "keine Kinder"
    elif n_kinder == 1:
        zusatz = "ein Kind"
    elif n_kinder == 2:
        zusatz = "zwei Kinder"
    else:
        raise NotImplementedError(f"n_kinder = {n_kinder}")
    ergebnisse_nach_kindern[n_kinder] = ergebnisse_nach_kindern[n_kinder].rename(
        columns={
            "kindergeld_tu": "Kindergeld, " + zusatz,
            "eink_st_tu": "Einkommensteuer, " + zusatz,
            "soli_st_tu": "Soli, " + zusatz,
        }
    )
    

KeyError: Index(['tu_id'], dtype='object')

In [ ]:
ergebnisse_nach_kindern[0]

## Erstellen der Gesamtbruttoeinkünfte von Steuersubjekten und eines DataFrames für Vergleiche

Wir summieren nun die Einkünfte eines Haushalts auf (das funktioniert nur, weil wir alle anderen
Einkünfte neben dem monatlichen Bruttolohn auf Null und die Einkünfte der Kinder auf fehlend
gesetzt haben).

In [ ]:
gesamteink_tu = daten_nach_kindern[0].groupby("tu_id")["bruttolohn_m"].sum() * 12
gesamteink_tu.name = "Gesamteinkünfte Steuersubjekt"
gesamteink_tu.head()

Nun fügen wir die Gesamteinkünfte mit den von GETTSIM erhaltenen Werten für Einkommensteuer,
Soli und Kindergeld für alle drei Haushaltstypen zusammen

In [ ]:
vergleich = gesamteink_tu.to_frame().join(ergebnisse_nach_kindern[0]).join(ergebnisse_nach_kindern[1]).join(ergebnisse_nach_kindern[2]).set_index("Gesamteinkünfte Steuersubjekt").round(2)
vergleich

## Berechnung des Steuervorteils aus der Einkommensteuer für Haushalte mit Kindern im Vergleich zu Haushalten ohne.

In [ ]:
for n_kinder in "ein Kind", "zwei Kinder":
    vergleich["Steuervorteil Einkommensteuer, " + n_kinder] = (
        vergleich["Einkommensteuer, keine Kinder"]
        - vergleich["Kindergeld, keine Kinder"]
        - vergleich["Einkommensteuer, " + n_kinder]
        + vergleich["Kindergeld, " + n_kinder]
    )
vergleich

## Plots Kindergeld, Einkommensteuer, Vorteil für Haushalte mit Kindern

In [ ]:
vergleich[[c for c in vergleich.columns if c.startswith("Kindergeld")]].plot();

In [ ]:
vergleich[[c for c in vergleich.columns if c.startswith("Eink")]].plot();

In [ ]:
vergleich[[c for c in vergleich.columns if c.startswith("Steuervorteil Einkommensteuer")]].plot();

In [ ]:
vergleich.loc[80000:].head(20)

## Aufgabe 1: Interpretation der Dualität Kindergeld / Kinderfreibetrag (20 Minuten)

Warum sind die Kurven für die Steuervorteile für Einkommen bis in den Bereich zwischen 80.000€ und 90.000€ flach?

*Hier Platz für Ihre Antwort*

Warum sind die Kurven flach für sehr hohe Einkommen?

*Hier Platz für Ihre Antwort*

Warum steigen die Kurven im Bereich dazwischen nahezu linear an?

*Hier Platz für Ihre Antwort*

## Aufgabe 2: Gesamter Steuervorteil (überspringen, falls sonst weniger als 10 Minuten für Aufgabe 3).


Wiederholen Sie die Berechnung des Steuervorteils von oben, beziehen Sie nun jedoch den Soli mit ein.

Erstellen Sie zwei Grafiken, wieder auf Ebene der Steuersubjekte: 

1. Solidaritätszuschlag nach Gesamteinkünften
2. Gesamter Steuervorteil nach Gesamteinkünften

Interpretieren Sie Ihre Ergebnisse (Sie dürfen gern Internetrecherche betreiben)!

In [ ]:
for n_kinder in "ein Kind", "zwei Kinder":
    vergleich["Steuervorteil gesamt, " + n_kinder] = (
        vergleich["Steuervorteil Einkommensteuer, " + n_kinder]
        + vergleich["Soli, keine Kinder"]
        - vergleich["Soli, " + n_kinder]
    )

In [ ]:
vergleich[[c for c in vergleich.columns if c.startswith("Soli")]].plot();

In [ ]:
vergleich[[c for c in vergleich.columns if c.startswith("Steuervorteil gesamt")]].plot();